Using SMAP data to plot different SSS years

In [1]:
import xarray as ax
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import ticker, cm

import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
# import cftime

import glob

In [2]:
#%matplotlib inline
%matplotlib qt5                         
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../SSS_PLOT/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
SSS_PLOT='png'

savefig = 1

In [3]:
path_dir = r"C:\Users\rolte\Desktop\MY RESEARCH ON GULF OF GUINEA\SSS SMAP\2017"

In [4]:
file_list = glob.glob(path_dir + "\RSS_smap_SSS_L3_8day_running_2017*FNL_v04.0.nc")

In [5]:
print(len(file_list))

365


In [6]:
# Use Xarray to open all at once, concatenate along a given dimension

# Open all files in file_list, concatenating along the "time" dimension. combine='nested' allows you to choose/specify which dimension
ds_all = ax.open_mfdataset(file_list,combine='nested',concat_dim='time', drop_variables=['gice','fland','gland','nobs','nobs_40km','sss_ref'])
print(ds_all)

#ds_all.lon,ds_all.lat,ds_all.sss_smap

<xarray.Dataset>
Dimensions:               (lon: 1440, lat: 720, time: 365)
Coordinates:
  * lon                   (lon) float32 0.125 0.375 0.625 ... 359.4 359.6 359.9
  * lat                   (lat) float32 -89.88 -89.62 -89.38 ... 89.62 89.88
  * time                  (time) datetime64[ns] 2017-01-01T12:00:00 ... 2017-...
Data variables:
    sss_smap              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sss_smap_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sss_smap_40km         (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    surtep                (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/58)
    Conventions:                                    CF-1.6, ACDD-1.3
    standard_name_vocabulary:                       CF Standard Name Table v27
    title:                                          SMAP ocean surfac

In [7]:
#Define plot dimension
Xo = 0
Yo = 0
X1 = 15
Y1 = 10
[xmin, xmax, ymin, ymax] = [Xo-X1, Xo+X1, Yo-Y1, Yo+Y1]
extent = [xmin, xmax, ymin, ymax]
#global_extent=True

In [8]:
ds_all=ds_all.sel(lat=slice(ymin,ymax))

In [9]:
#Opening the data in the variables
#Nobs = ds_all.nobs
#nobs_40km = ds_all.nobs_40km
#SSS = ds_all.sss_smap.values
#SST = ds_all.surtep.values
sss_40km = ds_all.sss_smap_40km.values
#sss_smap_uncertainty = ds_all.sss_smap_uncertainty




#ds_all.SST[time,:,:,]            #To specify the data to plot from the file list, the two semi column is for lon and lat, and the time is the stepping number

In [10]:
#Assigning a boolean value to scale
scale = True

In [11]:
def plot_variables(lon,lat,variable,ax,extent,title,levels,label,scale):
    
    
    if scale is True:
        scale_type = ticker.LogLocator()            #Setting the conditions for scale types
    else:
        scale_type = ticker.MaxNLocator()
        
    cs = ax.contourf(lon,lat,np.squeeze(variable), levels, extend='both', transform=ccrs.PlateCarree(), locator=scale_type)
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.4,.4,.3], edgecolor='black')
    ax.add_feature(cartopy.feature.RIVERS, zorder=3, edgecolor="blue")
    ax.add_feature(cartopy.feature.BORDERS, zorder=3, edgecolor=[.0,.10,.1])
    ax.add_feature(cartopy.feature.LAKES, zorder=3, edgecolor="blue")
    rd = plt.colorbar(cs,ax=ax, orientation='vertical', fraction = 0.04, extend='both')
    rd.set_label('Salinity (' + label+")", fontsize = 12)
    ax.set_title(title, fontsize = 10)
    
    plt.set_cmap(cmap=plt.get_cmap('turbo'))
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.ylabels_right = False
    
    return cs,rd,ax # function creates 3 outputs

In [12]:
#Subplot line of codes for SSS_SMAP

# Loop and plot 4 figures from all the lists
for t in range(2):
    plt.close('all')
    
    data_time = ds_all.time[t].values                             #Loading the time dataset
    day_str = np.datetime_as_string(data_time,unit='m')           #
    fig,ax = plt.subplots(1,1, subplot_kw = {'projection' : ccrs.PlateCarree(central_longitude=60)}, figsize = (10,12))

    levels = np.arange(25,38,.2)
    plot_variables(ds_all.lon,ds_all.lat,sss_40km[t,:,:],ax,extent, 'SMAP Sea Surface Salinity at Original 40km Resolution',levels, 'ppt',False)


    ax.text(5,7,day_str[0:10], transform=ccrs.PlateCarree(),zorder=10, fontsize = 10)         #To add a timestamp to the plot

    #To save the subplot as individual image
    if savefig:                              
        plt.savefig(__figdir__+'2017_SSS_'+str(t) +'.'+SSS_PLOT,**savefig_args)
        
        
#To add text in the plot body line of codes
#ax.text(x,y,'text')           while x and y is the dimension/position of the text in the plot

C:\Users\rolte\AppData\Local\Temp/ipykernel_21192/3528676352.py:15: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  rd = plt.colorbar(cs,ax=ax, orientation='vertical', fraction = 0.04, extend='both')
C:\Users\rolte\anaconda3\lib\site-packages\cartopy\mpl\gridliner.py:343: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


FileNotFoundError: [Errno 2] No such file or directory: '../SSS_PLOT/2017_SSS_0.png'